In [1]:
# ================================================
# ✅ 1️⃣ LIBRARIES & SETUP
# ================================================
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import re
import string

# ================================================
# ✅ 2️⃣ PATHS
# ================================================
image_dir = "/kaggle/input/basem/images"
input_csv = "/kaggle/input/basem/dataset.csv"

# ================================================
# ✅ 3️⃣ LOAD & PREPROCESS CSV
# ================================================
df = pd.read_csv(input_csv)

existing_data = []
for _, row in df.iterrows():
    image_filename = row['image_path']
    full_image_path = os.path.join(image_dir, image_filename)
    if os.path.exists(full_image_path):
        label_converted = row['label 2'] - 1
        existing_data.append({
            'Image_path': full_image_path,
            'Label_Sentiment': label_converted
        })

processed_df = pd.DataFrame(existing_data)

# ================================================
# ✅ 4️⃣ DATA SPLIT
# ================================================
train_df, temp_df = train_test_split(processed_df, test_size=0.3, stratify=processed_df['Label_Sentiment'], random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=1/3, stratify=temp_df['Label_Sentiment'], random_state=42)

# Add label column for consistency
for df_name, df_ in [('train', train_df), ('test', test_df), ('val', val_df)]:
    df_['label'] = df_['Label_Sentiment']
    df_.to_csv(f'/kaggle/working/{df_name}_vision_only.csv', index=False)

# ================================================
# ✅ 5️⃣ LOAD CLIP MODEL
# ================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# ================================================
# ✅ 6️⃣ VISION-ONLY DATASET
# ================================================
class VisionOnlyDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['Image_path']).convert('RGB')
        label = row['label']
        return image, label

def collate_fn(batch):
    images, labels = zip(*batch)
    images = list(images)
    labels = torch.tensor(labels)
    return images, labels

# ================================================
# ✅ 7️⃣ DATALOADERS
# ================================================
batch_size = 8

train_loader = DataLoader(VisionOnlyDataset(train_df), batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(VisionOnlyDataset(val_df), batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(VisionOnlyDataset(test_df), batch_size=batch_size, collate_fn=collate_fn)

# ================================================
# ✅ 8️⃣ VISION-ONLY CLASSIFICATION MODEL
# ================================================
class VisionClassifier(torch.nn.Module):
    def __init__(self, img_dim, num_classes=3):
        super().__init__()
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(img_dim, num_classes)

    def forward(self, img_feat):
        x = self.dropout(img_feat)
        logits = self.classifier(x)
        return logits

# ================================================
# ✅ 9️⃣ GET IMAGE DIMENSION + MODEL
# ================================================
dummy_image = Image.new('RGB', (224, 224))
dummy_img = clip_processor(images=dummy_image, return_tensors="pt").to(device)
img_dim = clip_model.get_image_features(**dummy_img).shape[1]

model = VisionClassifier(img_dim).to(device)

# ================================================
# ✅ 🔟 LOSS & OPTIMIZER
# ================================================
class_weights = train_df['label'].value_counts().sort_index().tolist()
total = sum(class_weights)
weights = [total / c for c in class_weights]
criterion = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(weights).to(device))
optimizer = AdamW(model.parameters(), lr=1e-4)

# ================================================
# ✅ 1️⃣1️⃣ TRAINING LOOP
# ================================================
num_epochs = 20
patience = 3
patience_counter = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # ============================================================
    # TRAINING PHASE
    # ============================================================
    model.train()
    total_train_loss = 0

    for images, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
        img_inputs = clip_processor(images=images, return_tensors="pt").to(device)
        img_feat = clip_model.get_image_features(**img_inputs)

        logits = model(img_feat)
        loss = criterion(logits, labels.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # ============================================================
    # VALIDATION PHASE
    # ============================================================
    model.eval()
    total_val_loss = 0
    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            image_inputs = clip_processor(images=images, return_tensors="pt").to(device)
            img_features = clip_model.get_image_features(**image_inputs)

            logits = model(img_features)
            loss = criterion(logits, labels.to(device))

            total_val_loss += loss.item()
            
            # Store predictions for metrics
            predictions = torch.argmax(logits, dim=1)
            val_predictions.extend(predictions.cpu().numpy())
            val_labels.extend(labels.numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_labels, val_predictions)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

    # ============================================================
    # EARLY STOPPING CHECK
    # ============================================================
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_vision_model.pt")
        print("✅ Validation loss improved — model saved.")
    else:
        patience_counter += 1
        print(f"⏰ No improvement — patience {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break

# ================================================
# ✅ 1️⃣2️⃣ FINAL TEST EVALUATION
# ================================================
print("\n🔍 Loading best model for final evaluation...")
model.load_state_dict(torch.load("best_vision_model.pt"))
model.eval()

test_predictions = []
test_labels = []
total_test_loss = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Final Test Evaluation"):
        image_inputs = clip_processor(images=images, return_tensors="pt").to(device)
        img_features = clip_model.get_image_features(**image_inputs)

        logits = model(img_features)
        loss = criterion(logits, labels.to(device))
        
        total_test_loss += loss.item()
        predictions = torch.argmax(logits, dim=1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.numpy())

# Calculate final metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_predictions, average='weighted')
cm = confusion_matrix(test_labels, test_predictions)

print("\n📊 FINAL TEST RESULTS (VISION-ONLY):")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test Loss: {total_test_loss/len(test_loader):.4f}")
print(f"\nConfusion Matrix:\n{cm}")

# ================================================
# ✅ 1️⃣3️⃣ ADDITIONAL METRICS PER CLASS
# ================================================
precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(test_labels, test_predictions, average=None)

print("\n📈 PER-CLASS METRICS:")
for i in range(len(precision_per_class)):
    print(f"Class {i}: Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, F1={f1_per_class[i]:.4f}, Support={support[i]}")

2025-07-07 08:53:57.656313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751878437.821887      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751878437.872251      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Epoch [19/20] Train Loss: 0.8081 | Val Loss: 0.8514 | Val Acc: 0.6475
⏰ No improvement — patience 1/3


Validation Epoch 20: 100%|██████████| 57/57 [00:17<00:00,  3.31it/s]


Epoch [20/20] Train Loss: 0.8056 | Val Loss: 0.8502 | Val Acc: 0.6563
✅ Validation loss improved — model saved.

🔍 Loading best model for final evaluation...


Final Test Evaluation: 100%|██████████| 113/113 [00:38<00:00,  2.96it/s]


📊 FINAL TEST RESULTS (VISION-ONLY):
Test Accuracy: 0.6619
Test Precision: 0.6960
Test Recall: 0.6619
Test F1-Score: 0.6673
Test Loss: 0.7830

Confusion Matrix:
[[246 114  42]
 [ 35 260  58]
 [ 11  45  91]]

📈 PER-CLASS METRICS:
Class 0: Precision=0.8425, Recall=0.6119, F1=0.7089, Support=402
Class 1: Precision=0.6205, Recall=0.7365, F1=0.6736, Support=353
Class 2: Precision=0.4764, Recall=0.6190, F1=0.5385, Support=147
